# Homework10 - datatalks.club - Rui Pinto

## Building the image

In [1]:
!docker pull svizor/zoomcamp-model:3.11.5-hw10

3.11.5-hw10: Pulling from svizor/zoomcamp-model

e7c4b030: Already exists 
36e84c8e: Already exists 
b60fbb0c: Already exists 
e97f4fbd: Already exists 
d4e17ae3: Already exists 
f8d7a667: Already exists 
cd59a5af: Already exists 
fa7bf560: Pulling fs layer 
9d6f76f0: Pulling fs layer 
baa659f2: Pulling fs layer 
c5660603: Pull complete  528B/528B1MBBADigest: sha256:fc4ca0a74559f57ef8ee4f8e3c3586160ab74427c7c01c8971e4d273db3a2e06
Status: Downloaded newer image for svizor/zoomcamp-model:3.11.5-hw10
docker.io/svizor/zoomcamp-model:3.11.5-hw10


## Question 1

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 zoomcamp-model:3.11.5-hw10
```
And in another terminal, execute q6_test.py file:

```python
python q6_test.py
```

You should see this:

```bash
{'has_subscribed': True, 'has_subscribed_probability': <value>}
```

Here <value> is the probability of getting a subscription. You need to choose the right one.

- 0.287
- 0.530
- 0.757
- 0.960

In [22]:
!docker images

REPOSITORY              TAG           IMAGE ID       CREATED      SIZE
kindest/node            <none>        2d9b4b74084a   5 days ago   1.05GB
svizor/zoomcamp-model   3.11.5-hw10   020904a2523c   7 days ago   548MB


In [24]:
#run in terminal:  docker run -it --rm -p 9696:9696 svizor/zoomcamp-model:3.11.5-hw10

In [25]:
!python q6_test.py

{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}


## Installing kubectl and kind

You need to install:

- kubectl - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
- kind - https://kind.sigs.k8s.io/docs/user/quick-start/

In [4]:
#checking version to install
!uname -m

x86_64


### Installing kubectl

In [9]:
#!curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
#!curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl.sha256"
#!echo "$(cat kubectl.sha256)  kubectl" | sha256sum --check

#in terminal: sudo install -o root -g root -m 0755 kubectl /usr/local/bin/kubectl

In [11]:
!kubectl version --client

Client Version: v1.32.0
Kustomize Version: v5.5.0


In [13]:
### installing kind
#in terminal: [ $(uname -m) = x86_64 ] && curl -Lo ./kind https://kind.sigs.k8s.io/dl/v0.26.0/kind-linux-amd64

In [14]:
# For AMD64 / x86_64
#[ $(uname -m) = x86_64 ] && curl -Lo ./kind https://kind.sigs.k8s.io/dl/v0.26.0/kind-linux-amd64
#chmod +x ./kind
#sudo mv ./kind /usr/local/bin/kind

zsh:1: command not found: kind


In [15]:
!kind --version

kind version 0.26.0


### Creating a cluster

Now let's create a cluster with kind:

```
kind create cluster
```

And check with kubectl that it was successfully created:

```
kubectl cluster-info
```


## Question 3

What's the smallest deployable computing unit that we can create and manage in Kubernetes (kind in our case)?

- Node
- **Pod**
- Deployment
- Service


Explanation of the Options:

- Node:
  - A Node is a physical or virtual machine that serves as a worker in the Kubernetes cluster. It runs Pods but is not the smallest deployable unit.

- Pod:
  - A Pod is the smallest deployable unit in Kubernetes. It represents one or more containers that are tightly coupled and share resources like storage and network.
  - You manage Pods using higher-level abstractions like Deployments, but the Pod itself is the basic unit of deployment.

- Deployment:
  - A Deployment is a higher-level abstraction that manages Pods. It ensures the desired number of Pods are running and can handle updates and rollbacks.

- Service:
  - A Service provides a stable network identity for Pods, enabling communication within or outside the cluster. It is not a deployable unit.

## Question 4
Now let's test if everything works. Use kubectl to get the list of running services.

What's the Type of the service that is already running there?

- NodePort
- **ClusterIP**
- ExternalName
- LoadBalancer

In [26]:
!kubectl get service #kubectl get svc

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   38m


## Question 5

To be able to use the docker image we previously created (zoomcamp-model:3.11.5-hw10), we need to register it with kind.

What's the command we need to run for that?
- kind create cluster 
- kind build node-image
- **kind load docker-image**
- kubectl apply

In [27]:
!docker ps

CONTAINER ID   IMAGE                               COMMAND                  CREATED          STATUS          PORTS                                       NAMES
73248a8d8096   svizor/zoomcamp-model:3.11.5-hw10   "waitress-serve --li…"   13 minutes ago   Up 13 minutes   0.0.0.0:9696->9696/tcp, :::9696->9696/tcp   cranky_cannon
5df01bc51490   kindest/node:v1.32.0                "/usr/local/bin/entr…"   39 minutes ago   Up 39 minutes   127.0.0.1:45919->6443/tcp                   kind-control-plane


In [28]:
!kind load docker-image svizor/zoomcamp-model:3.11.5-hw10

Image: "svizor/zoomcamp-model:3.11.5-hw10" with ID "sha256:020904a2523cec81c854a9b4679ae26d23f814e42350721136d1447f910a6b53" found to be already present on all nodes.


## Question 6
Now let's create a deployment config (e.g. deployment.yaml):

```YAML
apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  selector:
    matchLabels:
      app: subscription
  replicas: 1
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: <Image>
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: <Memory>
            cpu: <CPU>
        ports:
        - containerPort: <Port>
```

Replace <Image>, <Memory>, <CPU>, <Port> with the correct values.

What is the value for <Port>?

Apply this deployment using the appropriate command and get a list of running Pods. You can see one running Pod.

In [30]:
%%writefile deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  selector:
    matchLabels:
      app: subscription
  replicas: 1
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: svizor/zoomcamp-model:3.11.5-hw10
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: "128Mi"   
            cpu: "200m"       
        ports:
        - containerPort: 9696  

Writing deployment.yaml


In [31]:
!!ls -lh deployment.yaml

['-rw-r--r-- 1 ruifspinto ruifspinto 540 Dec 18 21:56 deployment.yaml']

In [32]:
!kubectl apply -f deployment.yaml

deployment.apps/subscription created


In [33]:
!kubectl get deploy,pod

NAME                           READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/subscription   1/1     1            1           12s

NAME                                READY   STATUS    RESTARTS   AGE
pod/subscription-695fd4dc9b-l9z5q   1/1     Running   0          12s


## Question 7
Let's create a service for this deployment (service.yaml):

```YAML
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```    
    
Fill it in. What do we need to write instead of <???>?

Apply this config file.

In [34]:
%%writefile service.yaml
apiVersion: v1
kind: Service
metadata:
  name: subscription
spec:
  type: LoadBalancer
  selector:
    app: subscription
  ports:
  - port: 80
    targetPort: 9696

Writing service.yaml


In [35]:
!ls -lh service.yaml

-rw-r--r-- 1 ruifspinto ruifspinto 164 Dec 18 21:57 service.yaml


In [36]:
!kubectl apply -f service.yaml

service/subscription created


In [37]:
!kubectl get svc

NAME           TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
kubernetes     ClusterIP      10.96.0.1       <none>        443/TCP        42m
subscription   LoadBalancer   10.96.114.133   <pending>     80:32458/TCP   7s


In [38]:
!kubectl describe svc subscription

Name:                     subscription
Namespace:                default
Labels:                   <none>
Annotations:              <none>
Selector:                 app=subscription
Type:                     LoadBalancer
IP Family Policy:         SingleStack
IP Families:              IPv4
IP:                       10.96.114.133
IPs:                      10.96.114.133
Port:                     <unset>  80/TCP
TargetPort:               9696/TCP
NodePort:                 <unset>  32458/TCP
Endpoints:                10.244.0.5:9696
Session Affinity:         None
External Traffic Policy:  Cluster
Internal Traffic Policy:  Cluster
Events:                   <none>


### Testing the service

In [40]:
#kubectl port-forward service/subscription 9696:80

In [52]:
!python q6_test.py

{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}


### Autoscaling
Now we're going to use a HorizontalPodAutoscaler (HPA for short) that automatically updates a workload resource (such as our deployment), with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

```bash
kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3
```

You can check the current status of the new HPA by running:

```bash
kubectl get hpa
```

The output should be similar to the next:

```bash
NAME               REFERENCE                 TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   1%/20%    1         3         1          27s
```

TARGET column shows the average CPU consumption across all the Pods controlled by the corresponding deployment. Current CPU consumption is about 0% as there are no clients sending requests to the server.

> Note: In case the HPA instance doesn't run properly, try to install the latest Metrics Server release from the components.yaml manifest:
> 
> ```kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml```

In [53]:
!kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3

Error from server (AlreadyExists): horizontalpodautoscalers.autoscaling "subscription-hpa" already exists


In [54]:
!kubectl get hpa

NAME               REFERENCE                 TARGETS              MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: <unknown>/20%   1         3         1          26m


In [58]:
!kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml

serviceaccount/metrics-server unchanged
clusterrole.rbac.authorization.k8s.io/system:aggregated-metrics-reader unchanged
clusterrole.rbac.authorization.k8s.io/system:metrics-server unchanged
rolebinding.rbac.authorization.k8s.io/metrics-server-auth-reader unchanged
clusterrolebinding.rbac.authorization.k8s.io/metrics-server:system:auth-delegator unchanged
clusterrolebinding.rbac.authorization.k8s.io/system:metrics-server unchanged
service/metrics-server unchanged
deployment.apps/metrics-server configured
apiservice.apiregistration.k8s.io/v1beta1.metrics.k8s.io unchanged


In [59]:
!kubectl get pods -n kube-system

NAME                                         READY   STATUS    RESTARTS   AGE
coredns-668d6bf9bc-q6hqm                     1/1     Running   0          85m
coredns-668d6bf9bc-z66vv                     1/1     Running   0          85m
etcd-kind-control-plane                      1/1     Running   0          85m
kindnet-zzznx                                1/1     Running   0          85m
kube-apiserver-kind-control-plane            1/1     Running   0          85m
kube-controller-manager-kind-control-plane   1/1     Running   0          85m
kube-proxy-2lwk2                             1/1     Running   0          85m
kube-scheduler-kind-control-plane            1/1     Running   0          85m
metrics-server-6f7dd4c4c4-b5qd8              0/1     Running   0          35m


In [56]:
!kubectl top nodes

error: Metrics API not available


In [57]:
!kubectl top pods

error: Metrics API not available


In [63]:
!kubectl describe hpa subscription-hpa

Name:                                                  subscription-hpa
Namespace:                                             default
Labels:                                                <none>
Annotations:                                           <none>
CreationTimestamp:                                     Wed, 18 Dec 2024 22:06:08 +0000
Reference:                                             Deployment/subscription
Metrics:                                               ( current / target )
  resource cpu on pods  (as a percentage of request):  <unknown> / 20%
Min replicas:                                          1
Max replicas:                                          3
Deployment pods:                                       1 current / 0 desired
Conditions:
  Type           Status  Reason                   Message
  ----           ------  ------                   -------
  AbleToScale    True    SucceededGetScale        the HPA controller was able to get the target's current sca

## Increase the load

Let's see how the autoscaler reacts to increasing the load. To do this, we can slightly modify the existing q6_test.py script by putting the operator that sends the request to the subscription service into a loop.

```python
while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)
```

Now you can run this script.


In [46]:
%%writefile q6_test_mod.py
import requests
from time import sleep
url = "http://localhost:9696/predict"
client = {"job": "management", "duration": 400, "poutcome": "success"}
while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)

Writing q6_test_mod.py


In [47]:
!python q6_test_mod.py

{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
{'has_subscribed': True, 'has_subscrib

## Question 8 (optional)

Run kubectl get hpa subscription-hpa --watch command to monitor how the autoscaler performs. Within a minute or so, you should see the higher CPU load; and then - more replicas. What was the maximum amount of the replicas during this test?

- 1
- 2
- **3**
- 4

> Note: It may take a few minutes to stabilize the number of replicas. Since the amount of load is not controlled in any way it may happen that the final number of replicas will differ from initial.

In [65]:
#!kubectl get hpa subscription-hpa --watch